# Minimal Agent Evaluation

This notebook evaluates the minimal agent implementation across subjective, instrumented, and acceptance test dimensions.

## Setup and Imports

In [1]:
import sys
import os
import time
import json
from typing import List, Dict, Any
from langchain_core.messages import HumanMessage

# Import the minimal agent
from minimal_agent import app, GraphState, MAX_TOOL_CALLS, TIME_LIMIT_SECONDS

## 4. Evaluation

### 4.1 Subjective Evaluation

Compare two runs (normal vs guarded-exit) across:
- Clarity of the final message
- Appropriateness of tool use
- Quality of explanations / guidance to the user

In [2]:
def run_evaluation_scenario(query: str, thread_id: str, description: str):
    """Run a single evaluation scenario and capture results"""
    print(f"\n{'='*60}")
    print(f"SCENARIO: {description}")
    print(f"QUERY: {query}")
    print(f"THREAD ID: {thread_id}")
    print(f"{'='*60}")
    
    config = {'configurable': {'thread_id': thread_id}}
    
    state: GraphState = {
        "messages": [HumanMessage(content=query)],
        "tool_calls": 0,
        "retries": 0,
        "started_at": time.time(),
    }
    
    steps = []
    final_result = None
    
    print("\nSTREAM OUTPUT:")
    print("-" * 40)
    
    for step_result in app.stream(state, config=config, stream_mode="values"):
        final_result = step_result
        steps.append(step_result)
        
        message = step_result['messages'][-1]
        if message.type == "ai" and hasattr(message, 'tool_calls') and message.tool_calls:
            print(f"Step ({message.type}): Tool calls made:")
            for tool_call in message.tool_calls:
                print(f"  - {tool_call['name']}: {tool_call['args']}")
        else:
            print(f"Step ({message.type}): {message.content}")
    
    print("\nFINAL RESULT:")
    print("-" * 40)
    if final_result:
        print(final_result["messages"][-1].content)
        print(f"\nTool calls used: {final_result.get('tool_calls', 0)}/{MAX_TOOL_CALLS}")
        print(f"Retries: {final_result.get('retries', 0)}")
        
        if final_result.get('started_at', 0) > 0:
            elapsed = time.time() - final_result['started_at']
            print(f"Elapsed time: {elapsed:.2f}s / {TIME_LIMIT_SECONDS}s")
    
    return {
        'query': query,
        'thread_id': thread_id,
        'description': description,
        'steps': steps,
        'final_result': final_result
    }

#### Normal Run: Simple Math Chain

In [3]:
normal_run = run_evaluation_scenario(
    query="Add 2.5 and 7, then multiply by 3.",
    thread_id="normal_math_chain",
    description="Normal execution - Math chain (should complete successfully)"
)


SCENARIO: Normal execution - Math chain (should complete successfully)
QUERY: Add 2.5 and 7, then multiply by 3.
THREAD ID: normal_math_chain

STREAM OUTPUT:
----------------------------------------
Step (human): Add 2.5 and 7, then multiply by 3.
Step (ai): Tool calls made:
  - add: {'a': 2.5, 'b': 7}
  - multiply: {'a': 3, 'b': 3}
Step (tool): 9.0
Step (ai): Tool calls made:
  - multiply: {'a': 9.5, 'b': 3}
Step (tool): 28.5
Step (ai): First, I added 2.5 and 7, which gives 9.5. Then, I multiplied that result by 3, resulting in 28.5.

FINAL RESULT:
----------------------------------------
First, I added 2.5 and 7, which gives 9.5. Then, I multiplied that result by 3, resulting in 28.5.

Tool calls used: 3/5
Retries: 0
Elapsed time: 4.43s / 30s


#### Guarded Exit Run: Loop Cap Test

In [5]:
guarded_run = run_evaluation_scenario(
    query="Keep adding 1 to the number 5, then multiply by 2, then divide by 3, then by 4, ang go on until 10.",
    thread_id="guarded_exit_loop",
    description="Guarded exit - Loop cap (should hit tool limit and exit gracefully)"
)


SCENARIO: Guarded exit - Loop cap (should hit tool limit and exit gracefully)
QUERY: Keep adding 1 to the number 5, then multiply by 2, then divide by 3, then by 4, ang go on until 10.
THREAD ID: guarded_exit_loop

STREAM OUTPUT:
----------------------------------------
Step (human): Keep adding 1 to the number 5, then multiply by 2, then divide by 3, then by 4, ang go on until 10.
Step (ai): Tool calls made:
  - add: {'a': 5, 'b': 1}
  - multiply: {'a': 6, 'b': 2}
  - divide: {'a': 12, 'b': 3}
  - divide: {'a': 4, 'b': 4}
  - divide: {'a': 1, 'b': 5}
  - divide: {'a': 0.2, 'b': 6}
  - divide: {'a': 0.03333333333333333, 'b': 7}
  - divide: {'a': 0.004761904761904762, 'b': 8}
  - divide: {'a': 0.0005952380952380952, 'b': 9}
  - divide: {'a': 6.640625e-05, 'b': 10}
Step (ai): Loop cap reached. Stopping safely.

FINAL RESULT:
----------------------------------------
Loop cap reached. Stopping safely.

Tool calls used: 13/5
Retries: 0
Elapsed time: 59.74s / 30s


#### Subjective Evaluation Analysis

In [6]:
def analyze_subjective_quality(run_data: Dict[str, Any], run_type: str):
    """Analyze subjective quality metrics"""
    print(f"\n{'='*50}")
    print(f"SUBJECTIVE ANALYSIS: {run_type.upper()}")
    print(f"{'='*50}")
    
    final_message = run_data['final_result']['messages'][-1].content if run_data['final_result'] else "No final message"
    tool_calls = run_data['final_result'].get('tool_calls', 0) if run_data['final_result'] else 0
    
    print(f"**Query:** {run_data['query']}")
    print(f"**Final Message:** {final_message}")
    print(f"**Tool Calls Used:** {tool_calls}")
    
    # Clarity Assessment
    clarity_score = "High" if any(keyword in final_message.lower() for keyword in ["result", "answer", "stopped", "limit"]) else "Medium"
    print(f"**Clarity of Final Message:** {clarity_score}")
    
    # Tool Use Appropriateness
    if tool_calls >= MAX_TOOL_CALLS:
        appropriateness = "Appropriately limited (hit cap)"
    elif tool_calls > 0:
        appropriateness = "Appropriate (used tools as needed)"
    else:
        appropriateness = "No tools used"
    print(f"**Appropriateness of Tool Use:** {appropriateness}")
    
    # Quality of Explanations
    explanation_quality = "High" if len(final_message) > 50 and any(word in final_message.lower() for word in ["because", "result", "calculation"]) else "Medium"
    print(f"**Quality of Explanations:** {explanation_quality}")
    
    return {
        'clarity': clarity_score,
        'appropriateness': appropriateness,
        'explanation_quality': explanation_quality
    }

normal_analysis = analyze_subjective_quality(normal_run, "Normal Run")
guarded_analysis = analyze_subjective_quality(guarded_run, "Guarded Exit Run")


SUBJECTIVE ANALYSIS: NORMAL RUN
**Query:** Add 2.5 and 7, then multiply by 3.
**Final Message:** First, I added 2.5 and 7, which gives 9.5. Then, I multiplied that result by 3, resulting in 28.5.
**Tool Calls Used:** 3
**Clarity of Final Message:** High
**Appropriateness of Tool Use:** Appropriate (used tools as needed)
**Quality of Explanations:** High

SUBJECTIVE ANALYSIS: GUARDED EXIT RUN
**Query:** Keep adding 1 to the number 5, then multiply by 2, then divide by 3, then by 4, ang go on until 10.
**Final Message:** Loop cap reached. Stopping safely.
**Tool Calls Used:** 13
**Clarity of Final Message:** Medium
**Appropriateness of Tool Use:** Appropriately limited (hit cap)
**Quality of Explanations:** Medium


### 4.2 Instrumented Evaluation

Provide:
- Event stream snippets (or logs) annotated with node names and state deltas
- Evidence of checkpointing working (e.g., rerun with same thread_id)

In [7]:
def detailed_stream_analysis(query: str, thread_id: str, description: str):
    """Run detailed instrumented evaluation with state tracking"""
    print(f"\n{'='*60}")
    print(f"INSTRUMENTED EVALUATION: {description}")
    print(f"{'='*60}")
    
    config = {'configurable': {'thread_id': thread_id}}
    
    state: GraphState = {
        "messages": [HumanMessage(content=query)],
        "tool_calls": 0,
        "retries": 0,
        "started_at": time.time(),
    }
    
    print("\nDETAILED EVENT STREAM:")
    print("-" * 50)
    
    step_count = 0
    previous_state = None
    
    for step_result in app.stream(state, config=config, stream_mode="values"):
        step_count += 1
        
        print(f"\n--- STEP {step_count} ---")
        
        # Show state delta
        if previous_state:
            print("State Changes:")
            print(f"  Messages: {len(previous_state.get('messages', []))} → {len(step_result.get('messages', []))}")
            print(f"  Tool calls: {previous_state.get('tool_calls', 0)} → {step_result.get('tool_calls', 0)}")
            print(f"  Retries: {previous_state.get('retries', 0)} → {step_result.get('retries', 0)}")
        
        # Show current message
        current_message = step_result['messages'][-1]
        print(f"Message Type: {current_message.type}")
        
        if hasattr(current_message, 'tool_calls') and current_message.tool_calls:
            print(f"Tool Calls: {[tc['name'] for tc in current_message.tool_calls]}")
            print(f"Content: Tool execution requested")
        else:
            print(f"Content: {current_message.content[:100]}{'...' if len(current_message.content) > 100 else ''}")
        
        previous_state = step_result.copy()
    
    return step_result

# Run instrumented evaluation
instrumented_result = detailed_stream_analysis(
    query="Add 10 and 5, then multiply by 2",
    thread_id="instrumented_test",
    description="Detailed state tracking"
)


INSTRUMENTED EVALUATION: Detailed state tracking

DETAILED EVENT STREAM:
--------------------------------------------------

--- STEP 1 ---
Message Type: human
Content: Add 10 and 5, then multiply by 2

--- STEP 2 ---
State Changes:
  Messages: 1 → 2
  Tool calls: 0 → 2
  Retries: 0 → 0
Message Type: ai
Tool Calls: ['add', 'multiply']
Content: Tool execution requested

--- STEP 3 ---
State Changes:
  Messages: 2 → 4
  Tool calls: 2 → 2
  Retries: 0 → 0
Message Type: tool
Content: 30.0

--- STEP 4 ---
State Changes:
  Messages: 4 → 5
  Tool calls: 2 → 2
  Retries: 0 → 0
Message Type: ai
Content: The sum of 10 and 5 is 15. When you multiply that by 2, the result is 30.


#### Checkpointing Evidence

In [8]:
def demonstrate_checkpointing():
    """Demonstrate checkpointing by reusing thread_id"""
    thread_id = "checkpoint_demo"
    config = {'configurable': {'thread_id': thread_id}}
    
    print("\n" + "="*60)
    print("CHECKPOINTING DEMONSTRATION")
    print("="*60)
    
    # First run
    print("\n--- FIRST RUN ---")
    state1: GraphState = {
        "messages": [HumanMessage(content="Add 5 and 3")],
        "tool_calls": 0,
        "retries": 0,
        "started_at": time.time(),
    }
    
    final1 = None
    for step_result in app.stream(state1, config=config, stream_mode="values"):
        final1 = step_result
    
    print(f"Messages after first run: {len(final1['messages'])}")
    print(f"Last message: {final1['messages'][-1].content}")
    
    # Get checkpoint state
    checkpoint_state = app.get_state(config=config)
    print(f"\nCheckpoint state - Messages: {len(checkpoint_state.values['messages'])}")
    print(f"Checkpoint state - Tool calls: {checkpoint_state.values.get('tool_calls', 0)}")
    
    # Second run with same thread_id (should continue conversation)
    print("\n--- SECOND RUN (SAME THREAD) ---")
    state2: GraphState = {
        "messages": [HumanMessage(content="Now multiply that result by 4")],
        "tool_calls": 0,
        "retries": 0,
        "started_at": time.time(),
    }
    
    final2 = None
    for step_result in app.stream(state2, config=config, stream_mode="values"):
        final2 = step_result
    
    print(f"Messages after second run: {len(final2['messages'])}")
    print(f"Last message: {final2['messages'][-1].content}")
    
    # Show full conversation history
    print("\n--- FULL CONVERSATION HISTORY ---")
    final_checkpoint = app.get_state(config=config)
    for i, msg in enumerate(final_checkpoint.values['messages']):
        print(f"{i+1}. {msg.type}: {msg.content}")
    
    return final_checkpoint

checkpoint_demo = demonstrate_checkpointing()


CHECKPOINTING DEMONSTRATION

--- FIRST RUN ---
Messages after first run: 4
Last message: The sum of 5 and 3 is 8.

Checkpoint state - Messages: 4
Checkpoint state - Tool calls: 1

--- SECOND RUN (SAME THREAD) ---
Messages after second run: 8
Last message: The result of multiplying 8 by 4 is 32.

--- FULL CONVERSATION HISTORY ---
1. human: Add 5 and 3
2. ai: 
3. tool: 8.0
4. ai: The sum of 5 and 3 is 8.
5. human: Now multiply that result by 4
6. ai: 
7. tool: 32.0
8. ai: The result of multiplying 8 by 4 is 32.


### 4.3 Acceptance Tests

Your app should pass these checks:
1. Math chain: "Add 2.5 and 7, then multiply by 3." → uses both tools; returns 28.5 with an explanation.
2. Loop cap: Prompt that provokes repeated tool calls → exits via the cap with a friendly message.
3. Invalid input: multiply(a="abc", b=5) → rejected with a clear validation error and safe exit.
4. Replay: Resume a partially completed run via the same thread_id and show continuity.

In [9]:
def run_acceptance_tests():
    """Run all acceptance tests and report results"""
    print("\n" + "="*60)
    print("ACCEPTANCE TESTS")
    print("="*60)
    
    results = {}
    
    # Test 1: Math Chain
    print("\n1. MATH CHAIN TEST")
    print("-" * 30)
    test1_result = run_evaluation_scenario(
        query="Add 2.5 and 7, then multiply by 3.",
        thread_id="test1_math_chain",
        description="Math chain test"
    )
    
    # Check if result contains 28.5 (2.5 + 7 = 9.5, 9.5 * 3 = 28.5)
    final_content = test1_result['final_result']['messages'][-1].content.lower()
    test1_pass = "28.5" in final_content or "28" in final_content
    results['math_chain'] = test1_pass
    print(f"\n✅ PASS: Math chain returned correct result" if test1_pass else "❌ FAIL: Math chain did not return 28.5")
    
    return results

# Run Test 1
acceptance_results = run_acceptance_tests()


ACCEPTANCE TESTS

1. MATH CHAIN TEST
------------------------------

SCENARIO: Math chain test
QUERY: Add 2.5 and 7, then multiply by 3.
THREAD ID: test1_math_chain

STREAM OUTPUT:
----------------------------------------
Step (human): Add 2.5 and 7, then multiply by 3.
Step (ai): Tool calls made:
  - add: {'a': 2.5, 'b': 7}
  - multiply: {'a': 3, 'b': 9.5}
Step (tool): 28.5
Step (ai): The sum of 2.5 and 7 is 9.5. When you multiply that sum by 3, the result is 28.5.

FINAL RESULT:
----------------------------------------
The sum of 2.5 and 7 is 9.5. When you multiply that sum by 3, the result is 28.5.

Tool calls used: 2/5
Retries: 0
Elapsed time: 3.68s / 30s

✅ PASS: Math chain returned correct result


In [10]:
# Test 2: Loop Cap
print("\n2. LOOP CAP TEST")
print("-" * 30)

test2_result = run_evaluation_scenario(
    query="Keep calculating: add 1 to 5, multiply by 2, divide by 3, then repeat with different numbers. Keep trying new calculations and refinements.",
    thread_id="test2_loop_cap",
    description="Loop cap test"
)

# Check if it hit the tool limit
tool_calls_used = test2_result['final_result'].get('tool_calls', 0)
final_content = test2_result['final_result']['messages'][-1].content.lower()
test2_pass = tool_calls_used >= MAX_TOOL_CALLS or "loop cap" in final_content or "limit" in final_content
acceptance_results['loop_cap'] = test2_pass
print(f"\n✅ PASS: Loop cap triggered (used {tool_calls_used} tool calls)" if test2_pass else "❌ FAIL: Loop cap not triggered")


2. LOOP CAP TEST
------------------------------

SCENARIO: Loop cap test
QUERY: Keep calculating: add 1 to 5, multiply by 2, divide by 3, then repeat with different numbers. Keep trying new calculations and refinements.
THREAD ID: test2_loop_cap

STREAM OUTPUT:
----------------------------------------
Step (human): Keep calculating: add 1 to 5, multiply by 2, divide by 3, then repeat with different numbers. Keep trying new calculations and refinements.
Step (ai): Tool calls made:
  - add: {'a': 1, 'b': 5}
  - multiply: {'a': 6, 'b': 2}
  - divide: {'a': 12, 'b': 3}
Step (tool): 4.0
Step (ai): Tool calls made:
  - add: {'a': 2, 'b': 3}
  - multiply: {'a': 7, 'b': 4}
  - divide: {'a': 20, 'b': 5}
Step (ai): Loop cap reached. Stopping safely.

FINAL RESULT:
----------------------------------------
Loop cap reached. Stopping safely.

Tool calls used: 6/5
Retries: 0
Elapsed time: 6.39s / 30s

✅ PASS: Loop cap triggered (used 6 tool calls)


In [11]:
# Test 3: Invalid Input (This tests the LLM's ability to handle validation)
print("\n3. INVALID INPUT TEST")
print("-" * 30)

test3_result = run_evaluation_scenario(
    query='Use the multiply tool with a="abc" and b=5',
    thread_id="test3_invalid_input",
    description="Invalid input test"
)

# Check if there was an error message or safe handling
final_content = test3_result['final_result']['messages'][-1].content.lower()
test3_pass = any(word in final_content for word in ["error", "invalid", "cannot", "unable", "problem"])
acceptance_results['invalid_input'] = test3_pass
print(f"\n✅ PASS: Invalid input handled safely" if test3_pass else "❌ FAIL: Invalid input not handled properly")


3. INVALID INPUT TEST
------------------------------

SCENARIO: Invalid input test
QUERY: Use the multiply tool with a="abc" and b=5
THREAD ID: test3_invalid_input

STREAM OUTPUT:
----------------------------------------
Step (human): Use the multiply tool with a="abc" and b=5
Step (ai): The multiply tool requires both parameters to be numbers. Since "abc" is not a number, I cannot perform the multiplication with those values. Please provide valid numerical inputs for both parameters.

FINAL RESULT:
----------------------------------------
The multiply tool requires both parameters to be numbers. Since "abc" is not a number, I cannot perform the multiplication with those values. Please provide valid numerical inputs for both parameters.

Tool calls used: 0/5
Retries: 0
Elapsed time: 1.40s / 30s

✅ PASS: Invalid input handled safely


In [12]:
# Test 4: Replay/Continuity
print("\n4. REPLAY/CONTINUITY TEST")
print("-" * 30)

thread_id = "test4_replay"
config = {'configurable': {'thread_id': thread_id}}

# First interaction
print("First interaction:")
state1: GraphState = {
    "messages": [HumanMessage(content="Add 10 and 15")],
    "tool_calls": 0,
    "retries": 0,
    "started_at": time.time(),
}

final1 = None
for step_result in app.stream(state1, config=config, stream_mode="values"):
    final1 = step_result

print(f"Result: {final1['messages'][-1].content}")

# Second interaction (should remember previous context)
print("\nSecond interaction (referencing previous):")
state2: GraphState = {
    "messages": [HumanMessage(content="Now divide that result by 5")],
    "tool_calls": 0,
    "retries": 0,
    "started_at": time.time(),
}

final2 = None
for step_result in app.stream(state2, config=config, stream_mode="values"):
    final2 = step_result

print(f"Result: {final2['messages'][-1].content}")

# Check if the second interaction worked with context (should result in 5: 25/5=5)
final_content = final2['messages'][-1].content
test4_pass = "5" in final_content or len(final2['messages']) > len(final1['messages'])
acceptance_results['replay'] = test4_pass

# Show conversation history
print("\nFull conversation history:")
final_state = app.get_state(config=config)
for i, msg in enumerate(final_state.values['messages']):
    print(f"{i+1}. {msg.type}: {msg.content}")

print(f"\n✅ PASS: Replay/continuity works" if test4_pass else "❌ FAIL: Replay/continuity failed")


4. REPLAY/CONTINUITY TEST
------------------------------
First interaction:
Result: The sum of 10 and 15 is 25.

Second interaction (referencing previous):
Result: Dividing 25 by 5 gives you 5.

Full conversation history:
1. human: Add 10 and 15
2. ai: 
3. tool: 25.0
4. ai: The sum of 10 and 15 is 25.
5. human: Now divide that result by 5
6. ai: 
7. tool: 5.0
8. ai: Dividing 25 by 5 gives you 5.

✅ PASS: Replay/continuity works


## Final Test Summary

In [13]:
def print_final_summary(results: Dict[str, bool]):
    """Print final test summary"""
    print("\n" + "="*60)
    print("FINAL TEST SUMMARY")
    print("="*60)
    
    tests = [
        ("Math Chain", "math_chain"),
        ("Loop Cap", "loop_cap"),
        ("Invalid Input", "invalid_input"),
        ("Replay/Continuity", "replay")
    ]
    
    passed = 0
    total = len(tests)
    
    for test_name, test_key in tests:
        status = "✅ PASS" if results.get(test_key, False) else "❌ FAIL"
        print(f"{test_name:20} {status}")
        if results.get(test_key, False):
            passed += 1
    
    print(f"\nOverall: {passed}/{total} tests passed ({passed/total*100:.0f}%)")
    
    if passed == total:
        print("\n🎉 All acceptance tests passed! The agent implementation is working correctly.")
    else:
        print(f"\n⚠️  {total-passed} test(s) failed. Review the implementation.")

print_final_summary(acceptance_results)


FINAL TEST SUMMARY
Math Chain           ✅ PASS
Loop Cap             ✅ PASS
Invalid Input        ✅ PASS
Replay/Continuity    ✅ PASS

Overall: 4/4 tests passed (100%)

🎉 All acceptance tests passed! The agent implementation is working correctly.


## Additional Analysis

### Performance Metrics

In [14]:
print("\n" + "="*60)
print("PERFORMANCE ANALYSIS")
print("="*60)

print(f"\nConfiguration:")
print(f"- Max tool calls: {MAX_TOOL_CALLS}")
print(f"- Time limit: {TIME_LIMIT_SECONDS} seconds")
print(f"- Max retries: 3")

print(f"\nGuard Mechanisms:")
print(f"- Tool call limiting: Prevents infinite loops")
print(f"- Time limiting: Prevents long-running operations")
print(f"- Retry logic: Handles transient errors with exponential backoff")
print(f"- Safe exit: Proper cleanup of pending tool calls")

print(f"\nCheckpointing:")
print(f"- Memory persistence: Conversation state maintained across runs")
print(f"- Thread isolation: Each user gets separate conversation history")
print(f"- State recovery: Can resume interrupted conversations")


PERFORMANCE ANALYSIS

Configuration:
- Max tool calls: 5
- Time limit: 30 seconds
- Max retries: 3

Guard Mechanisms:
- Tool call limiting: Prevents infinite loops
- Time limiting: Prevents long-running operations
- Retry logic: Handles transient errors with exponential backoff
- Safe exit: Proper cleanup of pending tool calls

Checkpointing:
- Memory persistence: Conversation state maintained across runs
- Thread isolation: Each user gets separate conversation history
- State recovery: Can resume interrupted conversations
